In [79]:
api_key

'509b1b28aacae6635c78ba106789f9c4'

In [80]:
date_start = "2005-09-01"
date_end = "2022-12-31"
issn = '1552-4450'

api_keys = pd.read_csv('data/apikeys.csv')
find_api_key = api_keys[api_keys['api_server']=='springer_eric'].reset_index()
global_api_key = find_api_key['api_key'][0]

# use this link:    https://dev.springernature.com/adding-constraints

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun May 17 17:24:43 2020
Search in Spring Nature using API by keywords for NIS lab reserch projects.

@author oudeng, Graduate School of Human Sciences, Waseda University

Python program for searching info from Springer Nature via API by XML.
Results includes the title, pdf url and abstract of destination articles. 

https://dev.springernature.com/signup.  
registerred by WasedaID of Ou,DENG, got necessary API.

How to use?
1) Run this program in Python eviroment, including bs4 and requests lib.
2) Input keywords for searching.
3) Searching results will in Python console window.
4) Read in console window directly if not too many results,
   or copy contents to any other more confortable browsers.
5) Copy url of artitle you like in results, use broswer to read PDF.
 
Can use for other API?
Yes. By modifying base_url, api_key, total and content identification tags.
Just confirm the url you try in other API. 

Can for JSON?
No, this programe for XML only.

"""

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import itertools

# Use a for-loop to flatten a list of lists
def flatten_lists(listToFlatten):
    flat_list = list()
    for sub_list in listToFlatten:
        flat_list += sub_list
    return flat_list

def getXML(url):
    try:
        r = requests.get(url, timeout = 30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "Scraping Errors. Oooop!!"

def spList(total): 
    S=1 
    P=50
    if total%P != 0:
        s = list(range(S, total//P*P, P)) + [total//P*P+1]
        p = list([P]*(len(s)-1)) + [total%P]
    else:
        s = list(range(S, total//P*P, P)) 
        p = list([P]*len(s))
    # print(s)
    # print(p)
    return s, p

def search_function():
    base_url = "http://api.springernature.com/metadata/pam?q="
    api_key = "&api_key=" + global_api_key
    #Please register at SpringerNature as the following url for your API key.
    # https://www.springernature.com/gp/campaign/librarian-covid-tdm?sap-outbound-id=64AF2B62DCE26C591DAA9263090CDAF763E0CD1F
    
    #input keywords to search in API, url_1 for confirm 'total' value    
    
    date_range = "onlinedatefrom:" + date_start + "%20 onlinedateto:" + date_end
    journalISSN = " issn: " + issn
    
    # ================================================================= change keywords
    #keywords = input("Input '+' in searching keywords:    ")
    # =================================================================
    
    url_1 = base_url + date_range + journalISSN + api_key
    xml_1 = getXML(url_1)
    soup = bs(xml_1,'html.parser')
    total = int(soup.find('total').string)
    
    print('Search Results Totally =',total)
    
    s, p = spList(total)
    
    full_list_of_papers = []
    for i in range(len(s)):
    # ================================================================= change keywords
        url = base_url + date_range + journalISSN + '&s=' + str(s[i]) + '&p=' + str(p[i]) + api_key
    # =================================================================
        xml = getXML(url)
        soup = bs(xml, "html.parser")
        current_list_of_papers = soup.find_all('pam:message')
        full_list_of_papers.append(current_list_of_papers)
    
            
    print('Total=',total)
    return flatten_lists(full_list_of_papers)

## pull data 
## get citation number 
#https://stackoverflow.com/questions/69067691/retrieve-number-of-citations-of-a-scientific-paper-in-a-given-year
# pull all the data
# get data from allometric 

def search_to_dataframe(flattened_papers_list):
    all_papers = flattened_papers_list
    first_paper = all_papers[0]
    list_of_fields = list(set([tag.name for tag in first_paper.find_all()]))

    ## create a dictionary with a set of keys that is column titles
    dict_of_paper_fields = {}
    for i in list_of_fields: 
        appenddict = {i:[]}
        dict_of_paper_fields.update(appenddict)

    ## find the fields for each paper
    for i in all_papers: 
        for j in list_of_fields: 
            dict_of_paper_fields[j].append(i.find_all(j))

    return pd.DataFrame(dict_of_paper_fields)

def csv_of_data(dataframeCreated):
    dataframeCreated.to_csv('data/' +issn+ "__"+ date_start+"_"+date_end+ '.csv',index=False)

def main():
    list_of_search = search_function()
    dataframe_of_search = search_to_dataframe(list_of_search)
    csv_of_data(dataframe_of_search)

In [81]:
date_start = "2005-09-01"
date_end = "2022-12-31"
issn = '1552-4450'
main()

c:\Users\erick\gitrepositories\convergence_oracle\.venv\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Search Results Totally = 3684
Total= 3684
